In [15]:
import os
import wandb
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import torch
from pandas import DataFrame
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
from transformers import EvalPrediction
from typing import Optional
from transformers import (
    Trainer,
    BertTokenizer,
    TrainingArguments,
    HfArgumentParser,
    BertForSequenceClassification,
    EarlyStoppingCallback
)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_START_METHOD"] = "thread"
wandb.init(project="hyper_demo", 
           tags=["baseline", "hyperparam"],
           group="group_2")


source_data_path = "/data/tmp/nlp-data/open_source_data/classification/ChnSentiCorp_htl_all.csv"
ber_name_path = "/data/tmp/christmas.wang/chinese_wwm_ext_pytorch"

wandb: ERROR Control-C detected -- Run data was not synced


In [2]:
source_df = pd.read_csv(source_data_path, encoding="utf-8")
source_df.dropna(how="any", axis=0, inplace=True)
source_df

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
source_df = shuffle(source_df)
source_df

,label,review
6973,0,一、总体来讲，尚可。因为是2007年新开业的酒店，各项硬件设施还能将就。二、浴室及卫生间的设...
1143,1,注意啦！！这个酒店只有6、7楼重新装修过（地毯是深灰色）。其他都是旧装修。我们出住时，提出要...
6174,0,在携程订威龙原本看上了两点，一个是四星级别、一个是香港公司管理。住过之后发现名不副实。首先服...
2883,1,"比较高的性价比,环境服务都不错.68的自助餐也还不错,每人给10块钱的抵现券,58/人.就是..."
1301,1,"不错,就是院子小了一些,到海滩需要走一段端小路,奇怪的是青岛所有酒店都没有自己的海滩"
...,...,...
1200,1,"我已经是三年第三次通过携程预订这间酒店了,这次有些失望,酒店的服务态度还是很好,无可挑剔,可..."
4464,1,房间蛮合心意，早餐也很丰富，附属设施一般。周围的环境比较差，不过还算有几个小吃店能对付一顿简餐。
17,1,酒店正在申定五星中，不过目前看来四星都有点勉强。大堂很气派，不过细节很粗糙。硬件需要加强。服...
932,1,服务算可以设施老旧空调太吵在济南，也没别的好选择


In [4]:
eval_df = source_df[0:1600]
eval_df

,label,review
6973,0,一、总体来讲，尚可。因为是2007年新开业的酒店，各项硬件设施还能将就。二、浴室及卫生间的设...
1143,1,注意啦！！这个酒店只有6、7楼重新装修过（地毯是深灰色）。其他都是旧装修。我们出住时，提出要...
6174,0,在携程订威龙原本看上了两点，一个是四星级别、一个是香港公司管理。住过之后发现名不副实。首先服...
2883,1,"比较高的性价比,环境服务都不错.68的自助餐也还不错,每人给10块钱的抵现券,58/人.就是..."
1301,1,"不错,就是院子小了一些,到海滩需要走一段端小路,奇怪的是青岛所有酒店都没有自己的海滩"
...,...,...
763,1,五月底入住的该宾馆，对于三星来说还是可以的，价钱不贵，可能它的位置不是繁华地吧。就是前台的收...
7651,0,其他没什么，就是房价便宜。很老的星级宾馆了，愧对4星的标准了。补充点评2008年5月31日：...
5001,1,酒店房间很好，浴室在房间中间，但要小心一定要把窗帘拉下，对面就是办公楼。酒店服务非常到位。绝...
2400,1,酒店设施虽然有很多创新，但是并不为使用者着想的。窗帘自动开关虽然好，但是却不太好用。还有控制...


In [5]:
train_df = source_df[1600:-1]
train_df

,label,review
1618,1,"酒店离火车站不远,打的只要8元。就在金山寺边上，人工湖还和风景区相通，可惜没找到入口，否则省..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2278,1,酒店门脸有点小，你们倒是很大，房间还不错！就是房间的门已经有些下沉了，关门一定要用点力量，早...
4045,1,还可以，离汽车站也很近。离轮渡也很近，也比较干净。起先我预定了厦门元利酒店，结果上网还有时间...
6623,0,手机已收到短消息回复订房OK，前台自己没查到，却叫我打电话给携程，要我自己搞定，即使我给前台...
...,...,...
6968,0,"地理位置比较方便,但酒店服务态度不好,环境普通"
1200,1,"我已经是三年第三次通过携程预订这间酒店了,这次有些失望,酒店的服务态度还是很好,无可挑剔,可..."
4464,1,房间蛮合心意，早餐也很丰富，附属设施一般。周围的环境比较差，不过还算有几个小吃店能对付一顿简餐。
17,1,酒店正在申定五星中，不过目前看来四星都有点勉强。大堂很气派，不过细节很粗糙。硬件需要加强。服...


In [6]:
le = LabelEncoder()
le.fit(source_df["label"].tolist())

LabelEncoder()

In [7]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [8]:
def df_2_dataset(df: DataFrame, le: LabelEncoder, text: str, label: str, tokenizer) -> Dataset:
    """Transform DataFrame into DataSet

    Args:
        df (DataFrame): Source Data
        le (LabelEncoder): LabelEncoder
        text (str): Column Name of Text
        label (str): Column Name of Label
        tokenizer ([type]): Tokenizer

    Returns:
        Dataset: DataSet
    """
    x = list(df[text])
    df["label_id"] = le.transform(df[label].tolist())
    y = list(df["label_id"])
    x_tokenized = tokenizer(x, padding=True, truncation=True, max_length=512)
    result = Dataset(x_tokenized, y)
    return result

In [9]:
tokenizer = BertTokenizer.from_pretrained(ber_name_path, do_lower_case=False)

In [10]:
train_dataset = df_2_dataset(train_df, le, "review", "label", tokenizer)
eval_dataset = df_2_dataset(eval_df, le, "review", "label", tokenizer)

/tmp/ipykernel_25336/3498692374.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label_id"] = le.transform(df[label].tolist())


In [11]:
def compute_metrics(results: EvalPrediction) -> Optional[dict]:
    """Calculate Metrics

    Args:
        results (EvalPrediction): Predictions & labels

    Returns:
        [type]: Dictionary of Metrics
    """
    pred, labels = results.predictions, results.label_ids
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="binary")
    precision = precision_score(y_true=labels, y_pred=pred, average="binary")
    f1 = f1_score(y_true=labels, y_pred=pred, average="binary")
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
def train(train_dataset: Dataset,
          valid_dataset: Dataset,
          pre_model_path: str
          ) -> None:
    args = TrainingArguments(
        report_to="wandb",
        output_dir="/data/christmas.wang/project/classification_base_project/output",
        evaluation_strategy="steps",
        eval_steps=500,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        num_train_epochs=6,
        seed=0,
        load_best_model_at_end=True,
    )
    
    def model_init():
        return BertForSequenceClassification.from_pretrained(pre_model_path, num_labels=2)
               
    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
    def hp_space(trial):
        return {
            "learning_rate": trial.suggest_float("learning_rate",1e-5, 5e-5, log=True),
            "weight_decay": trial.suggest_float("weight_decay", 0.01, 0.3),
            "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 8),
            "seed": trial.suggest_int("seed", 20, 40),
            "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
        }
    best_trial = trainer.hyperparameter_search(
        direction="maximize",
        backend="optuna",
        n_trials=10,
        hp_space=hp_space
    )
    print("*************************************")
    print(" Best run %s" % str(best_trial))
    print("*************************************")

In [14]:
    train(train_dataset,
          eval_dataset,
          ber_name_path)

PyTorch: setting up devices
loading configuration file /data/tmp/christmas.wang/chinese_wwm_ext_pytorch/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

loading weights file /data/tmp/christmas.wang/chinese_wwm_ext_pytorch/pytorch_model.bin
Some weights of the model checkp

wandb: Network error resolved after 0:01:35.727296, resuming normal operation.
wandb: Network error resolved after 0:03:37.356086, resuming normal operation.


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


[W 2021-10-18 11:56:45,713] Trial 0 failed because of the following error: RuntimeError('Caught RuntimeError in replica 1 on device 1.\nOriginal Traceback (most recent call last):\n  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker\n    output = module(*input, **kwargs)\n  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl\n    return forward_call(*input, **kwargs)\n  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1524, in forward\n    outputs = self.bert(\n  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl\n    return forward_call(*input, **kwargs)\n  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 990, in forward\n 

RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 1524, in forward
    outputs = self.bert(
  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/Miniconda3/envs/wf_pytorch_env/lib/python3.8/site-packages/transformers/models/bert/modeling_bert.py", line 990, in forward
    encoder_outputs = self.encoder(
  File "/data/Miniconda3/envs/wf_pytorch_env/lib/py

wandb: Network error (ConnectTimeout), entering retry loop.
